# T1136.001 - Local Account
Adversaries may create a local account to maintain access to victim systems. Local accounts are those configured by an organization for use by users, remote support, services, or for administration on a single system or service. With a sufficient level of access, the <code>net user /add</code> command can be used to create a local account. On macOS systems the <code>dscl -create</code> command can be used to create a local account.

Such accounts may be used to establish secondary credentialed access that do not require persistent remote access tools to be deployed on the system.

## Atomic Tests

### Atomic Test #1 - Create a user account on a Linux systemCreate a user via useradd
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
useradd -M -N -r -s /bin/bash -c evil_account evil_user
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 1

#### Cleanup: 
```bash
userdel evil_user
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Create a user account on a MacOS systemCreates a user on a MacOS system with dscl
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
dscl . -create /Users/evil_user
dscl . -create /Users/evil_user UserShell /bin/zsh
dscl . -create /Users/evil_user RealName "Evil Account"
dscl . -create /Users/evil_user UniqueID "1010"
dscl . -create /Users/evil_user PrimaryGroupID 80
dscl . -create /Users/evil_user NFSHomeDirectory /Users/evil_user
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 2

#### Cleanup: 
```bash
dscl . -delete /Users/evil_user
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Create a new user in a command promptCreates a new user in a command prompt. Upon execution, "The command completed successfully." will be displayed. To verify the
new account, run "net user" in powershell or CMD and observe that there is a new user named "T1136.001_CMD"
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
net user /add "T1136.001_CMD" "T1136.001_CMD!"
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 3

#### Cleanup: 
```cmd
net user /del "T1136.001_CMD" >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Create a new user in PowerShellCreates a new user in PowerShell. Upon execution, details about the new account will be displayed in the powershell session. To verify the
new account, run "net user" in powershell or CMD and observe that there is a new user named "T1136.001_PowerShell"
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
New-LocalUser -Name "T1136.001_PowerShell" -NoPassword
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 4

#### Cleanup: 
```powershell
Remove-LocalUser -Name "T1136.001_PowerShell" -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Create a new user in Linux with `root` UID and GID.Creates a new user in Linux and adds the user to the `root` group. This technique was used by adversaries during the Butter attack campaign.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
useradd -g 0 -M -d /root -s /bin/bash butter
if [ $(cat /etc/os-release | grep -i 'Name="ubuntu"') ]; then echo "butter:BetterWithButter" | sudo chpasswd; else echo "BetterWithButter" | passwd --stdin butter; fi;
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 5

#### Cleanup: 
```bash
userdel butter
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Create a new Windows admin userCreates a new admin user in a command prompt.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
net user /add "T1136.001_Admin" "T1136_pass"
net localgroup administrators "T1136.001_Admin" /add
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 6

#### Cleanup: 
```cmd
net user /del "T1136.001_Admin" >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1136.001 -TestNumbers 6 -Cleanup

## Detection
Monitor for processes and command-line parameters associated with local account creation, such as <code>net user /add</code> , <code>useradd</code> , and <code>dscl -create</code> . Collect data on account creation within a network. Event ID 4720 is generated when a user account is created on a Windows system. (Citation: Microsoft User Creation Event) Perform regular audits of local system accounts to detect suspicious accounts that may have been created by an adversary.